In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

/Volumes/sd4mac/app/anaconda3/lib/python3.7/site-packages/matplotlib/font_manager.py:232: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [2]:
import keras
from keras.preprocessing.text import Tokenizer
import sklearn
import sklearn.preprocessing

Using TensorFlow backend.


In [3]:
import pandas as pd

In [4]:
df_train = pd.read_csv('data/ad1/ch_ad.csv')

In [6]:
data_list = df_train['text']


In [7]:
class_list = df_train['label']

In [8]:
y_labels = list(class_list.value_counts().index)

In [9]:
label_encoder = sklearn.preprocessing.LabelEncoder()

In [10]:
label_encoder.fit(y_labels)

LabelEncoder()

In [11]:
num_labels = len(y_labels)

In [12]:
y_labels = keras.utils.to_categorical(class_list.map(lambda x: label_encoder.transform([x])), num_labels)

In [13]:
tokenizer = Tokenizer(filters='!"#$%&()*,-./:;<=>?@[\\]^_`{|}~\t\n',lower=True,split=" ")

In [15]:
def preprocess_text(texts):
    import jieba
    return texts.map(lambda x:' '.join(list(jieba.cut(x))))
texts = preprocess_text(data_list)
    

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/x0/cz_5x6r56m1_ql7wr_0f72w80000gn/T/jieba.cache
Loading model cost 1.158 seconds.
Prefix dict has been built succesfully.


In [16]:
tokenizer.fit_on_texts(texts)

In [17]:
vocab = tokenizer.word_index


In [18]:
def clean_chinese_str(s):
    out = ''
    #mystr = string.decode('utf-8')
    mystr = s
    mystr = mystr[:100]
    for letter in mystr:
        out += letter + ' '
    return out

In [19]:
len(df_train)

15586

In [20]:
train_count = int(len(df_train) * 0.9)

In [21]:
train_count

14027

In [22]:
X_train = texts[0:train_count]
Y_train = y_labels[0:train_count]
X_test = texts[train_count:]
Y_test = y_labels[train_count:]

In [23]:
x_train_word_ids = tokenizer.texts_to_sequences(X_train)

x_test_word_ids = tokenizer.texts_to_sequences(X_test)

In [24]:
x_train_padded_seqs = keras.preprocessing.sequence.pad_sequences(x_train_word_ids, padding='post', truncating='post', maxlen=100)

In [25]:
x_test_padded_seqs = keras.preprocessing.sequence.pad_sequences(x_test_word_ids,padding='post', truncating='post', maxlen=100)

In [26]:
x_train_padded_seqs[0]

array([ 1603,    40,   417, 14389,   530,   330,   950,   931,   110,
        4499,  4500,  4501, 14390,  2498,  1604,  5154,     6,  1437,
          23,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0], dtype=int32)

In [27]:
import numpy as np
x_train_padded_seqs=np.expand_dims(x_train_padded_seqs,axis=2)

In [29]:
x_test_padded_seqs=np.expand_dims(x_test_padded_seqs,axis=2)

In [30]:
model = keras.models.Sequential()
model.add(keras.layers.LSTM(256, dropout=0.5, recurrent_dropout=0.1))

In [31]:
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dense(2, activation='softmax'))


In [122]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [123]:
model.fit(x_train_padded_seqs, Y_train, batch_size=32, epochs=12, validation_data=(x_test_padded_seqs, Y_test))

Instructions for updating:
Use tf.cast instead.
Train on 14027 samples, validate on 1559 samples
Epoch 1/12
14027/14027 [==============================] - 106s 8ms/step - loss: 0.6713 - acc: 0.5568 - val_loss: 0.7875 - val_acc: 0.7915
Epoch 2/12
14027/14027 [==============================] - 101s 7ms/step - loss: 0.6598 - acc: 0.5701 - val_loss: 0.9723 - val_acc: 0.4195
Epoch 3/12
14027/14027 [==============================] - 101s 7ms/step - loss: 0.6510 - acc: 0.6065 - val_loss: 0.7444 - val_acc: 0.6985
Epoch 4/12
14027/14027 [==============================] - 106s 8ms/step - loss: 0.6159 - acc: 0.6613 - val_loss: 0.7693 - val_acc: 0.6113
Epoch 5/12
14027/14027 [==============================] - 101s 7ms/step - loss: 0.6067 - acc: 0.6674 - val_loss: 0.7638 - val_acc: 0.6671
Epoch 6/12
14027/14027 [==============================] - 103s 7ms/step - loss: 0.6051 - acc: 0.6618 - val_loss: 0.8045 - val_acc: 0.5568
Epoch 7/12
14027/14027 [==============================] - 103s 7ms/step - l

In [32]:
model2 = keras.models.Sequential()


In [33]:
#model2.add(keras.layers.Input(shape=(100,), dtype='int32'))
model2.add(keras.layers.Embedding(len(tokenizer.word_index)+1, 300, input_length=100))
model2.add(keras.layers.LSTM(256, dropout=0.5, recurrent_dropout=0.1))
model2.add(keras.layers.Dense(256, activation='relu'))
model2.add(keras.layers.Dense(2, activation='softmax'))


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [34]:
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [35]:
X_train = keras.preprocessing.sequence.pad_sequences(x_train_word_ids, padding='post', truncating='post', maxlen=100)
X_test = keras.preprocessing.sequence.pad_sequences(x_test_word_ids, padding='post', truncating='post', maxlen=100)

In [146]:
model2.fit(X_train, Y_train, batch_size=32, epochs=12, validation_data=(X_test
                                                                                 , Y_test))

Train on 14027 samples, validate on 1559 samples
Epoch 1/12
14027/14027 [==============================] - 267s 19ms/step - loss: 0.6865 - acc: 0.5492 - val_loss: 0.8714 - val_acc: 0.0000e+00
Epoch 2/12
14027/14027 [==============================] - 245s 17ms/step - loss: 0.5014 - acc: 0.7168 - val_loss: 0.5684 - val_acc: 0.7389
Epoch 3/12
14027/14027 [==============================] - 250s 18ms/step - loss: 0.1328 - acc: 0.9591 - val_loss: 0.8022 - val_acc: 0.6863
Epoch 4/12
14027/14027 [==============================] - 255s 18ms/step - loss: 0.0671 - acc: 0.9795 - val_loss: 1.0065 - val_acc: 0.6985
Epoch 5/12
14027/14027 [==============================] - 240s 17ms/step - loss: 0.0381 - acc: 0.9894 - val_loss: 0.6628 - val_acc: 0.8127
Epoch 6/12
14027/14027 [==============================] - 238s 17ms/step - loss: 0.0249 - acc: 0.9934 - val_loss: 0.9199 - val_acc: 0.7415
Epoch 7/12
14027/14027 [==============================] - 222s 16ms/step - loss: 0.0199 - acc: 0.9949 - val_loss:

In [36]:
model3 = keras.models.Sequential()
embedding_layer = keras.layers.Embedding(len(tokenizer.word_index)+1, 300, input_length=100, trainable=False, weights=[])

In [37]:
model3.add(embedding_layer)
model3.add(keras.layers.LSTM(256, dropout=0.5, recurrent_dropout=0.1))
model3.add(keras.layers.Dense(256, activation='relu'))
model3.add(keras.layers.Dense(2, activation='softmax'))

ValueError: You called `set_weights(weights)` on layer "embedding_2" with a  weight list of length 0, but the layer was expecting 1 weights. Provided weights: []...